In [1]:
import random
from research.weight_estimation.gtsf_data.gtsf_dataset import GTSFDataset
from research.weight_estimation.gtsf_data.gtsf_augmentation import GTSFAugmentation

import os
import math
import numpy as np
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.optimizers import RMSprop
from keras.models import load_model
import keras

from matplotlib import pyplot as plt

Using TensorFlow backend.


<h1> Load GTSF Data </h1>

In [2]:
akpd_scorer_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/keypoint-detection-scorer/akpd_scorer_model_TF.h5'
gtsf_dataset = GTSFDataset('2019-03-01', '2020-02-10', akpd_scorer_url)
df = gtsf_dataset.get_prepared_dataset()



Raw dataframe loaded!
Dataset preparation beginning...
3D spatial information added!
Adding AKPD scores...
Converting world keypoints to matrix form...
(19565, 14)


<h1> Perform Augmentation </h1>

In [8]:
df = df[(df.captured_at < '2019-09-20') & (df.median_depth < 1.0) & (df.akpd_score > 0.9)]
gtsf_augmentation = GTSFAugmentation(df)
y_bounds, jitter, trials = (0.5, 2.0), 10, 10
augmented_df = gtsf_augmentation.generate_augmented_dataset(y_bounds, jitter, trials, random_seed=0)
print(augmented_df.shape)

(134260, 14)


<h1> Create Train / Test Split </h1>

In [20]:
# select train / test sets such that there are no overlapping fish IDs

train_pct = 0.8
fish_ids = augmented_df.fish_id.unique().tolist()
train_fish_ids = random.choices(fish_ids, k=int(len(fish_ids) * train_pct))
train_mask = augmented_df.fish_id.isin(train_fish_ids)
test_mask = ~train_mask
X_train = augmented_df[train_mask].wkps.values
X_train = np.concatenate(X_train).reshape(X_train.shape[0], 24)
y_train = augmented_df[train_mask].weight.values
X_test = augmented_df[test_mask].wkps.values
X_test = np.concatenate(X_test).reshape(X_test.shape[0], 24)
y_test = augmented_df[test_mask].weight.values


<h1> Train Neural Network in Keras </h1>

In [26]:
inputs = Input(shape=(24,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(256, activation='relu')(inputs)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
pred = Dense(1)(x)
model = Model(input=inputs, output=pred)


/home/user/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [27]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=['accuracy'])
model.fit(X_train, 1e-4 * y_train, validation_data=(X_test, 1e-4 * y_test), batch_size=25, epochs=100)

Train on 77510 samples, validate on 56750 samples
Epoch 1/100
77510/77510 [==============================] - 9s 115us/step - loss: 0.0097 - accuracy: 0.0000e+00 - val_loss: 0.0034 - val_accuracy: 0.0000e+00
Epoch 2/100
77510/77510 [==============================] - 9s 114us/step - loss: 0.0031 - accuracy: 0.0000e+00 - val_loss: 0.0038 - val_accuracy: 0.0000e+00
Epoch 3/100
77510/77510 [==============================] - 9s 113us/step - loss: 0.0027 - accuracy: 0.0000e+00 - val_loss: 0.0024 - val_accuracy: 0.0000e+00
Epoch 4/100
77510/77510 [==============================] - 9s 113us/step - loss: 0.0024 - accuracy: 0.0000e+00 - val_loss: 0.0022 - val_accuracy: 0.0000e+00
Epoch 5/100
77510/77510 [==============================] - 9s 113us/step - loss: 0.0024 - accuracy: 0.0000e+00 - val_loss: 0.0019 - val_accuracy: 0.0000e+00
Epoch 6/100
77510/77510 [==============================] - 9s 114us/step - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0021 - val_accuracy: 0.0000e+00
Epoch 7/

77510/77510 [==============================] - 9s 111us/step - loss: 0.0016 - accuracy: 0.0000e+00 - val_loss: 0.0016 - val_accuracy: 0.0000e+00
Epoch 53/100
77510/77510 [==============================] - 9s 111us/step - loss: 0.0016 - accuracy: 0.0000e+00 - val_loss: 0.0015 - val_accuracy: 0.0000e+00
Epoch 54/100
77510/77510 [==============================] - 9s 111us/step - loss: 0.0016 - accuracy: 0.0000e+00 - val_loss: 0.0016 - val_accuracy: 0.0000e+00
Epoch 55/100
77510/77510 [==============================] - 9s 112us/step - loss: 0.0015 - accuracy: 0.0000e+00 - val_loss: 0.0016 - val_accuracy: 0.0000e+00
Epoch 56/100
77510/77510 [==============================] - 9s 113us/step - loss: 0.0015 - accuracy: 0.0000e+00 - val_loss: 0.0015 - val_accuracy: 0.0000e+00
Epoch 57/100
77510/77510 [==============================] - 9s 112us/step - loss: 0.0015 - accuracy: 0.0000e+00 - val_loss: 0.0017 - val_accuracy: 0.0000e+00
Epoch 58/100
77510/77510 [==============================] - 9s 11

In [31]:
y_pred = model.predict(X_test).squeeze()